In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import timm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from PIL import Image

# === FOLDER PATHS ===
ROOT_DATA_DIR = r"C:\Users\LENOVO\OneDrive\Desktop\Project"
ROIS_DIR = os.path.join(ROOT_DATA_DIR, "CLASSIFICATION_ROIS_128x128")
MODEL_SAVE_PATH = os.path.join(ROOT_DATA_DIR, "best_efficientnet_b4.pth")

# TRAINING CONSTANTS
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TARGET_SIZE = (128, 128)
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
print("✅ Libraries Loaded, Device:", DEVICE)


✅ Libraries Loaded, Device: cpu


In [2]:
class PCBDefectDataset(Dataset):
    def __init__(self, file_list, labels, transform=None):
        self.file_list = file_list
        self.labels = labels
        self.transform = transform
        self.label_map = {label: i for i, label in enumerate(sorted(list(set(labels))))}

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        image = Image.open(img_path).convert("RGB")
        label_str = self.labels[idx]
        label = self.label_map[label_str]
        if self.transform:
            image = self.transform(image)
        return image, label


def load_data_paths():
    all_files = glob.glob(os.path.join(ROIS_DIR, "*", "*.png"))
    all_labels = [os.path.basename(os.path.dirname(f)) for f in all_files]
    return all_files, all_labels


def get_transforms(is_train=True):
    if is_train:
        return transforms.Compose([
            transforms.Resize(TARGET_SIZE),
            transforms.RandomRotation(15),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.1, contrast=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize(TARGET_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
print("✅ Dataset class and helper functions defined")


✅ Dataset class and helper functions defined


In [3]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_acc = 0.0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(num_epochs):
        model.train()
        running_loss, running_corrects, total_train = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total_train += labels.size(0)

        epoch_loss = running_loss / total_train
        epoch_acc = running_corrects.double() / total_train

        # --- Validation ---
        model.eval()
        val_loss, val_corrects, total_val = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)
                total_val += labels.size(0)

        val_epoch_loss = val_loss / total_val
        val_epoch_acc = val_corrects.double() / total_val

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print("✅ Model saved (new best accuracy)")

        history["train_loss"].append(epoch_loss)
        history["train_acc"].append(epoch_acc.item())
        history["val_loss"].append(val_epoch_loss)
        history["val_acc"].append(val_epoch_acc.item())

    return history, best_acc

print("✅ Train function defined")


✅ Train function defined


In [4]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_acc = 0.0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(num_epochs):
        model.train()
        running_loss, running_corrects, total_train = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total_train += labels.size(0)

        epoch_loss = running_loss / total_train
        epoch_acc = running_corrects.double() / total_train

        # --- Validation ---
        model.eval()
        val_loss, val_corrects, total_val = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)
                total_val += labels.size(0)

        val_epoch_loss = val_loss / total_val
        val_epoch_acc = val_corrects.double() / total_val

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print("✅ Model saved (new best accuracy)")

        history["train_loss"].append(epoch_loss)
        history["train_acc"].append(epoch_acc.item())
        history["val_loss"].append(val_epoch_loss)
        history["val_acc"].append(val_epoch_acc.item())

    return history, best_acc

print("✅ Train function defined")


✅ Train function defined


In [5]:
files, labels = load_data_paths()
print(f"Total ROI images found: {len(files)}")

if not files:
    print("⚠️ No ROI images found. Check CLASSIFICATION_ROIS_128x128 folder.")
else:
    train_files, val_files, train_labels, val_labels = train_test_split(
        files, labels, test_size=0.2, stratify=labels, random_state=42
    )
    print("Training samples:", len(train_files))
    print("Validation samples:", len(val_files))
    print("Classes:", set(labels))

    train_dataset = PCBDefectDataset(train_files, train_labels, transform=get_transforms(is_train=True))
    val_dataset = PCBDefectDataset(val_files, val_labels, transform=get_transforms(is_train=False))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=len(set(labels)))
    model = model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    print("✅ Model, optimizer, and data loaders ready")


Total ROI images found: 3057
Training samples: 2445
Validation samples: 612
Classes: {'Open_circuit', 'Spurious_copper', 'Missing_hole', 'Spur', 'Short', 'Mouse_bite'}
✅ Model, optimizer, and data loaders ready


In [6]:
history, best_val_acc = train_model(model, criterion, optimizer, train_loader, val_loader, NUM_EPOCHS)
print(f"\n🎯 Training completed — Best Validation Accuracy: {best_val_acc:.4f}")


Epoch 1/10 | Train Loss: 1.7670, Train Acc: 0.4082, Val Loss: 1.1644, Val Acc: 0.5915
✅ Model saved (new best accuracy)
Epoch 2/10 | Train Loss: 0.8230, Train Acc: 0.7006, Val Loss: 0.7794, Val Acc: 0.7549
✅ Model saved (new best accuracy)
Epoch 3/10 | Train Loss: 0.4872, Train Acc: 0.8249, Val Loss: 0.4000, Val Acc: 0.8578
✅ Model saved (new best accuracy)
Epoch 4/10 | Train Loss: 0.3383, Train Acc: 0.8830, Val Loss: 0.2781, Val Acc: 0.9085
✅ Model saved (new best accuracy)
Epoch 5/10 | Train Loss: 0.2583, Train Acc: 0.9088, Val Loss: 0.2136, Val Acc: 0.9314
✅ Model saved (new best accuracy)
Epoch 6/10 | Train Loss: 0.1948, Train Acc: 0.9321, Val Loss: 0.1950, Val Acc: 0.9363
✅ Model saved (new best accuracy)
Epoch 7/10 | Train Loss: 0.1632, Train Acc: 0.9378, Val Loss: 0.2170, Val Acc: 0.9395
✅ Model saved (new best accuracy)
Epoch 8/10 | Train Loss: 0.1323, Train Acc: 0.9501, Val Loss: 0.1703, Val Acc: 0.9592
✅ Model saved (new best accuracy)
Epoch 9/10 | Train Loss: 0.1215, Train A